In [3]:
from PIL import Image
import os
os.environ['HF_ENDPOINT']='https://hf-mirror.com'
os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True'
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO']='0.0'
import argparse
import json
import time
import torch

from collections import defaultdict

from surya.input.langs import replace_lang_with_code
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.model import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings

import sys

import numpy as np
import pandas as pd
def process_image_bw(image):
    def convert_color(image,hit_colors,target_color,reverse=False):

        image_array = np.array(image)
        for color in hit_colors:
            expre = ((image_array[:, :, 0] == color[0]) & (image_array[:, :, 1] == color[1]) & (image_array[:, :, 2] == color[2]))
            if reverse:
                expre = ~expre
            image_array[expre] = target_color
        processed_image = Image.fromarray(image_array)
        return processed_image
    base_green = [0,40,32]
    base_red = [47,18,0]
    gap =5
    colors = []
    for i in range(gap):
        for j in range(gap):
            this_color = [base_green[0],base_green[1]+i,base_green[2]+j]
            minus_color = [base_green[0],base_green[1]-i,base_green[2]-j]
            colors.append(this_color)
            colors.append(minus_color)
            this_color = [base_red[0]+j,base_red[1]+i,base_red[2]]
            minus_color = [base_red[0]-j,base_red[1]-i,base_red[2]]
            colors.append(this_color)
            colors.append(minus_color)
    colors.extend([[0,0,0],[49,19,0],[0,68,54],[46,18,0],[68,27,12],[82,39,23],[0,68,54],[15,78,67]])

    new_image=convert_color(image,colors,target_color=[0,0,0])
    new_image=convert_color(new_image,[[0,0,0]],target_color=[255,255,255],reverse=True)
    image_array = np.array(new_image)
    white_expre = ((image_array[:, :, 0] == 255) & (image_array[:, :, 1] == 255) & (image_array[:, :, 2] == 255))
    black_expre = ((image_array[:, :, 0] == 0) & (image_array[:, :, 1] == 0) & (image_array[:, :, 2] == 0))
    image_array[white_expre] = [0,0,0]
    image_array[black_expre] = [255,255,255]
    new_image = Image.fromarray(image_array)
    return new_image
input_path = "/Volumes/dataset1/data/traderecord/eng"
max  = None
start_page = 0
images, names, _ = load_from_folder(input_path, max, start_page)

processed_path = os.path.join(input_path,"processed")
os.makedirs(processed_path, exist_ok=True)

for idx,image in enumerate(images):
    new_image = process_image_bw(image)
    file_save_path = os.path.join(processed_path,names[idx])
    print(file_save_path)
    new_image.save(file_save_path+".png")



/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_35_25
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_34_45
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_35_57
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_36_26
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_35_10
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_34_14
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_46_59
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_48_05
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_35_44
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_33_05
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_33_30
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_34_57
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_34_33
/Volumes/dataset1/data/traderecord/eng/processed/截屏2025-01-05 17_34_01
